# Creating new features

In [4]:
import pandas as pd
import numpy as np
import pickle
import warnings 
warnings.filterwarnings('ignore')

In [2]:
matches = pickle.load(open('data_extraction_1','rb'))
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,West Indies,India,0.1,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground
2,2,West Indies,India,0.3,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
3,2,West Indies,India,0.4,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
4,2,West Indies,India,0.5,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Auckland,Eden Park
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,Eden Park
120,964,Pakistan,New Zealand,19.4,2,0,Auckland,Eden Park
121,964,Pakistan,New Zealand,19.5,1,0,Auckland,Eden Park


In [ ]:
# We are going to create some new feature 

In [4]:
# required columns

# batting team
# bowling team
# current run rate
# wickets left
# city
# venue
# last five overs run
# ball left

In [5]:
matches.isna().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8919
venue                  0
dtype: int64

In [6]:
# As you can see there are some missing values in city columns so we are going to replace those values with venue

In [7]:
matches[matches['city'].isna()]['venue'].value_counts()  # we can use 1st word of venue to replace those values as a city

venue
Dubai International Cricket Stadium        3340
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [8]:
cities = np.where(matches['city'],matches['venue'].str.split().apply(lambda x: x[0]),matches['city'])  # 
# it will use 1st word of venue as city

In [9]:
matches['city'] = cities

In [10]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,West Indies,India,0.1,0,0,Central,Central Broward Regional Park Stadium Turf Ground
1,2,West Indies,India,0.2,0,JD Campbell,Central,Central Broward Regional Park Stadium Turf Ground
2,2,West Indies,India,0.3,0,0,Central,Central Broward Regional Park Stadium Turf Ground
3,2,West Indies,India,0.4,1,0,Central,Central Broward Regional Park Stadium Turf Ground
4,2,West Indies,India,0.5,0,0,Central,Central Broward Regional Park Stadium Turf Ground
...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Eden,Eden Park
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Eden,Eden Park
120,964,Pakistan,New Zealand,19.4,2,0,Eden,Eden Park
121,964,Pakistan,New Zealand,19.5,1,0,Eden,Eden Park


In [11]:
matches.drop(columns = 'venue', inplace = True)

In [12]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,West Indies,India,0.1,0,0,Central
1,2,West Indies,India,0.2,0,JD Campbell,Central
2,2,West Indies,India,0.3,0,0,Central
3,2,West Indies,India,0.4,1,0,Central
4,2,West Indies,India,0.5,0,0,Central
...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Eden
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Eden
120,964,Pakistan,New Zealand,19.4,2,0,Eden
121,964,Pakistan,New Zealand,19.5,1,0,Eden


In [13]:
matches['city'].value_counts()   # these are the number of balls in each city 3933/120 = total matches

city
Shere      3933
Eden       3499
Dubai      3340
R          2983
New        2819
           ... 
Sardar      121
Subrata     121
Hagley      121
Grange      121
Carrara      64
Name: count, Length: 90, dtype: int64

In [14]:
# we are going to consider those cities where number of marches is greater than 5 i.e number of balls greater than 600

In [14]:
matches['city'].value_counts()[matches['city'].value_counts()>600].shape   # unique cities where atleast five matches played



(34,)

In [15]:
matches['city'].value_counts()[matches['city'].value_counts()>600].sum()  # 51286 these data points will left after removing less matches cities





51286

In [16]:
a = list(matches['city'].value_counts()[matches['city'].value_counts()>600].index)

In [17]:
matches = matches[matches['city'].isin(a)]

In [18]:
matches.shape

(51286, 7)

In [20]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,West Indies,India,0.1,0,0,Central
1,2,West Indies,India,0.2,0,JD Campbell,Central
2,2,West Indies,India,0.3,0,0,Central
3,2,West Indies,India,0.4,1,0,Central
4,2,West Indies,India,0.5,0,0,Central
...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Eden
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Eden
120,964,Pakistan,New Zealand,19.4,2,0,Eden
121,964,Pakistan,New Zealand,19.5,1,0,Eden


In [23]:
matches['current_score'] = matches.groupby('match_id')['runs'].cumsum() # calculating current_score for each match 
# after each match current_score will set to zero because we use groupby on match id for defining match and runs for cumsum()


In [24]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,West Indies,India,0.1,0,0,Central,0
1,2,West Indies,India,0.2,0,JD Campbell,Central,0
2,2,West Indies,India,0.3,0,0,Central,0
3,2,West Indies,India,0.4,1,0,Central,1
4,2,West Indies,India,0.5,0,0,Central,1
...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Eden,149
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Eden,149
120,964,Pakistan,New Zealand,19.4,2,0,Eden,151
121,964,Pakistan,New Zealand,19.5,1,0,Eden,152


In [25]:
# creating column by spliting ball column for over and ball_no

In [26]:
matches['over'] = matches['ball'].apply(lambda x: str(x).split('.')[0])
matches['ball_no'] = matches['ball'].apply(lambda x: str(x).split('.')[1])

In [27]:
# Creating ball_left 
matches["balls_left"] = 120-(matches['over'].astype(int)*6 + matches["ball_no"].astype(int))

In [28]:
matches["balls_left"] = matches["balls_left"].apply(lambda x: 0 if x<0 else 0)   # if there os any extra ball in over



In [29]:
matches["balls_bowled"] = (matches['over'].astype(int)*6 + matches["ball_no"].astype(int))

In [30]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled
0,2,West Indies,India,0.1,0,0,Central,0,0,1,0,1
1,2,West Indies,India,0.2,0,JD Campbell,Central,0,0,2,0,2
2,2,West Indies,India,0.3,0,0,Central,0,0,3,0,3
3,2,West Indies,India,0.4,1,0,Central,1,0,4,0,4
4,2,West Indies,India,0.5,0,0,Central,1,0,5,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Eden,149,19,2,0,116
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Eden,149,19,3,0,117
120,964,Pakistan,New Zealand,19.4,2,0,Eden,151,19,4,0,118
121,964,Pakistan,New Zealand,19.5,1,0,Eden,152,19,5,0,119


In [31]:
matches['player_dismissed'] = matches['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)

In [32]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled
0,2,West Indies,India,0.1,0,0,Central,0,0,1,0,1
1,2,West Indies,India,0.2,0,1,Central,0,0,2,0,2
2,2,West Indies,India,0.3,0,0,Central,0,0,3,0,3
3,2,West Indies,India,0.4,1,0,Central,1,0,4,0,4
4,2,West Indies,India,0.5,0,0,Central,1,0,5,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Eden,149,19,2,0,116
119,964,Pakistan,New Zealand,19.3,0,1,Eden,149,19,3,0,117
120,964,Pakistan,New Zealand,19.4,2,0,Eden,151,19,4,0,118
121,964,Pakistan,New Zealand,19.5,1,0,Eden,152,19,5,0,119


In [33]:
matches['player_dismissed'] = matches.groupby('match_id')['player_dismissed'].cumsum()

In [34]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled
0,2,West Indies,India,0.1,0,0,Central,0,0,1,0,1
1,2,West Indies,India,0.2,0,1,Central,0,0,2,0,2
2,2,West Indies,India,0.3,0,1,Central,0,0,3,0,3
3,2,West Indies,India,0.4,1,1,Central,1,0,4,0,4
4,2,West Indies,India,0.5,0,1,Central,1,0,5,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,8,Eden,149,19,2,0,116
119,964,Pakistan,New Zealand,19.3,0,9,Eden,149,19,3,0,117
120,964,Pakistan,New Zealand,19.4,2,9,Eden,151,19,4,0,118
121,964,Pakistan,New Zealand,19.5,1,9,Eden,152,19,5,0,119


In [35]:
matches['wickets_left'] = 10-matches['player_dismissed']

In [36]:
matches

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left
0,2,West Indies,India,0.1,0,0,Central,0,0,1,0,1,10
1,2,West Indies,India,0.2,0,1,Central,0,0,2,0,2,9
2,2,West Indies,India,0.3,0,1,Central,0,0,3,0,3,9
3,2,West Indies,India,0.4,1,1,Central,1,0,4,0,4,9
4,2,West Indies,India,0.5,0,1,Central,1,0,5,0,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,8,Eden,149,19,2,0,116,2
119,964,Pakistan,New Zealand,19.3,0,9,Eden,149,19,3,0,117,1
120,964,Pakistan,New Zealand,19.4,2,9,Eden,151,19,4,0,118,1
121,964,Pakistan,New Zealand,19.5,1,9,Eden,152,19,5,0,119,1


In [37]:
#calculating current run rate score     
matches['crr'] = (matches['current_score']*6)/matches['balls_bowled']

In [38]:
matches['runs'].astype(int)

0      0
1      0
2      0
3      1
4      0
      ..
118    1
119    0
120    2
121    1
122    1
Name: runs, Length: 51286, dtype: int64

In [43]:
groups = matches.groupby('match_id')
match_id = matches['match_id'].unique()
last_five = []
for id in match_id:
    last_five.extend(groups.get_group(id).rolling(window = 30)['runs'].sum().values.tolist())
    # window 30 which means 5*6 = 30 balls in five overs
    

In [44]:
last_five

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 28.0,
 28.0,
 28.0,
 28.0,
 28.0,
 28.0,
 27.0,
 27.0,
 27.0,
 26.0,
 26.0,
 26.0,
 25.0,
 25.0,
 26.0,
 22.0,
 16.0,
 17.0,
 18.0,
 19.0,
 17.0,
 17.0,
 15.0,
 15.0,
 15.0,
 16.0,
 20.0,
 21.0,
 15.0,
 16.0,
 16.0,
 16.0,
 16.0,
 17.0,
 16.0,
 16.0,
 12.0,
 12.0,
 12.0,
 12.0,
 13.0,
 14.0,
 14.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0,
 20.0,
 19.0,
 25.0,
 26.0,
 26.0,
 26.0,
 26.0,
 26.0,
 22.0,
 22.0,
 24.0,
 23.0,
 23.0,
 24.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 25.0,
 21.0,
 24.0,
 24.0,
 24.0,
 23.0,
 23.0,
 29.0,
 24.0,
 24.0,
 25.0,
 25.0,
 25.0,
 24.0,
 30.0,
 30.0,
 28.0,
 28.0,
 28.0,
 27.0,
 26.0,
 25.0,
 25.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,

In [48]:
matches['last_five'] = last_five  # on each match for last 30 balls we get nan values as no sum values for past 30 balls

In [49]:
matches.head(1)

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left,crr,last_five
0,2,West Indies,India,0.1,0,0,Central,0,0,1,0,1,10,0.0,NaN


In [50]:
matches.groupby('match_id')['runs'].sum().reset_index().rename(columns = {'runs':'total_runs'})  # dataframe

,match_id,total_runs
0,2,95
1,3,150
2,4,202
3,5,111
4,6,191
...,...,...
418,956,180
419,957,211
420,958,138
421,959,222


In [51]:
final_df = matches.groupby('match_id')['runs'].sum().reset_index().rename(columns = {'runs':'total_runs'}).merge(matches , on ='match_id')  # merge on match_id




In [52]:
final_df

,match_id,total_runs,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_left,balls_bowled,wickets_left,crr,last_five
0,2,95,West Indies,India,0.1,0,0,Central,0,0,1,0,1,10,0.000000,NaN
1,2,95,West Indies,India,0.2,0,1,Central,0,0,2,0,2,9,0.000000,NaN
2,2,95,West Indies,India,0.3,0,1,Central,0,0,3,0,3,9,0.000000,NaN
3,2,95,West Indies,India,0.4,1,1,Central,1,0,4,0,4,9,1.500000,NaN
4,2,95,West Indies,India,0.5,0,1,Central,1,0,5,0,5,9,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51281,964,153,Pakistan,New Zealand,19.2,1,8,Eden,149,19,2,0,116,2,7.706897,56.0
51282,964,153,Pakistan,New Zealand,19.3,0,9,Eden,149,19,3,0,117,1,7.641026,50.0
51283,964,153,Pakistan,New Zealand,19.4,2,9,Eden,151,19,4,0,118,1,7.677966,52.0
51284,964,153,Pakistan,New Zealand,19.5,1,9,Eden,152,19,5,0,119,1,7.663866,53.0


In [53]:
final_df = final_df[['batting_team', 'bowling_team','city', 'current_score', 'crr', 'balls_left','wickets_left','last_five','total_runs']]

In [54]:
final_df.isna().sum()

batting_team         0
bowling_team         0
city                 0
current_score        0
crr                  0
balls_left           0
wickets_left         0
last_five        12227
total_runs           0
dtype: int64

In [55]:
final_df.dropna(inplace = True)

In [78]:
final_df.shape

(39059, 9)

In [57]:
final_df = final_df.sample(final_df.shape[0])   # shuffling

In [79]:
final_df.head(20)

,batting_team,bowling_team,city,current_score,crr,balls_left,wickets_left,last_five,total_runs
25296,Pakistan,Australia,Dubai,76,6.705882,0,9,39.0,151
8992,Sri Lanka,India,Punjab,176,9.962264,0,3,35.0,206
2425,Bangladesh,West Indies,Shere,206,10.474576,0,6,55.0,211
12772,England,Pakistan,Old,56,8.195122,0,9,48.0,135
9412,England,New Zealand,Seddon,37,7.400000,0,8,37.0,194
37373,Bangladesh,Pakistan,Gaddafi,42,5.142857,0,7,24.0,136
20401,New Zealand,Bangladesh,Bay,61,5.809524,0,7,20.0,194
1414,England,Afghanistan,R,73,7.551724,0,8,45.0,196
19053,Pakistan,South Africa,Dubai,51,4.191781,0,5,26.0,98
26533,Australia,South Africa,New,88,6.683544,0,5,29.0,166


In [81]:
final_df.to_csv('t20_cleandata.csv', index = False)  # saving dataset

# Creting model and pipeline

In [61]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [62]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse = False,drop = 'first'),['batting_team' ,'bowling_team','city'])
                     ]
,remainder = 'passthrough')







In [64]:
trf

ColumnTransformer(remainder='passthrough',
                  transformers=[('trf',
                                 OneHotEncoder(drop='first', sparse=False),
                                 ['batting_team', 'bowling_team', 'city'])])

In [74]:
# Total_runs is our y ind the rest is x
x = final_df.drop(columns=['total_runs'])
y = final_df['total_runs']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [75]:
pip = Pipeline(steps = [('step1', trf),
                        ('step2',StandardScaler()),
                        ('step3',XGBRegressor(n_estimator = 1000, max_depth = 12, learning_rate = 0.3,random_state = 1)
                                                                  
                                                                     
                                                                  )])

In [76]:
pip.fit(x_train,y_train)
y_pred = pip.predict(x_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9868894560731101
1.8680182025790275


In [57]:
pickle.dump(pip,open('pip.pkl_1','wb'))

In [115]:
list(final_df['city'].unique())

['Newlands',
 'Beausejour',
 'Kensington',
 'Shere',
 'Trent',
 'Wankhede',
 'Eden',
 'Dubai',
 'Warner',
 'Old',
 'Kennington',
 'Pallekele',
 'The',
 'Vidarbha',
 'Zahur',
 'Adelaide',
 'New',
 "Queen's",
 'SuperSport',
 'R.Premadasa',
 'Westpac',
 'Sydney',
 'R',
 "Lord's",
 'M',
 'Sophia',
 'Kingsmead',
 'Punjab',
 'Seddon',
 'Gaddafi',
 'Sheikh',
 'Central',
 'Bay',
 'Melbourne']

In [116]:
a = ['Newlands',
 'Beausejour',
 'Kensington',
 'Shere',
 'Trent',
 'Wankhede',
 'Eden',
 'Dubai',
 'Warner',
 'Old',
 'Kennington',
 'Pallekele',
 'The',
 'Vidarbha',
 'Zahur',
 'Adelaide',
 'New',
 "Queen's",
 'SuperSport',
 'R.Premadasa',
 'Westpac',
 'Sydney',
 'R',
 "Lord's",
 'M',
 'Sophia',
 'Kingsmead',
 'Punjab',
 'Seddon',
 'Gaddafi',
 'Sheikh',
 'Central',
 'Bay',
 'Melbourne']




In [ ]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'    
]